# Análisis de la base de datos de Synergy Logistics

### Análisis desarrollado por Daniel Amieva Rdz
#### El repositorio de este proyecto se encuentra en: https://github.com/Duque-de-Sealand/emtechinstitute-proyecto2

El proceso de mi análisis es el siguiente: primero escribo código que me ayude a entender la problemática y elaboro gráficas que se que me ayudarán a ilustrar mis conclusiones. 
Después, escribo mis conclusiones al final del documento.

Cargo las librerias que use para mi analísis

In [2]:
import pandas as pd
from siuba import * 
import altair as alt

Cargo la base de datos con read_csv y configuro las variables para que no sean simples objetivos string.
Veo rapidamente con head() que todo se haya cargado bien.

In [3]:

synergy_data = pd.read_csv("synergy_logistics_database.csv",
 delimiter = ",",
 dtype= {
     "register_id":"float",
     "direction":"category",
     "origin":"category",
     "destination":"category",
     "year":"float",
     "product":"category",
     "transport_mode":"category",
     "company_name":"category",
     "total_value":"float"
 } ,
  parse_dates=["date"])

synergy_data.head()

,register_id,direction,origin,destination,year,date,product,transport_mode,company_name,total_value
0,1.0,Exports,Japan,China,2015.0,2015-01-31,Cars,Sea,Honda,33000000.0
1,2.0,Exports,Japan,China,2015.0,2015-01-02,Cars,Sea,Honda,16000000.0
2,3.0,Exports,Japan,China,2015.0,2015-02-02,Cars,Sea,Honda,29000000.0
3,4.0,Exports,Japan,China,2015.0,2015-03-02,Cars,Sea,Honda,14000000.0
4,5.0,Exports,Japan,China,2015.0,2015-04-02,Cars,Sea,Honda,17000000.0


## Analisis preliminar con SweetViz

In [5]:
import sweetviz as sv
my_report = sv.analyze(synergy_data, target_feat = "total_value")
my_report.show_html(
    layout = "vertical",
    open_browser=True
)

#report revels interesting associations between total_value and particular categoricals
#plot a heatmap to justify the weigth put on company name
#count product by compnay
#count everything over time
#identificar a las companias mas valiosas y transport mode y direction y origin
#total value per export/import

                                             |                                             | [  0%]   00:00 ->…

Report SWEETVIZ_REPORT.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


El reporte muestra que se tiene 19,056 registros para analizar. El reporte tambien revela una fuerte asociacion entre país, dirección de envio y producto. Esto quiere decir que solo ciertos países venden ciertos productos y utilizan ciertas rutas. Esto es muy importante a considerar porque queremos asegurarnos que la decision que tomemos considere los productos que generan mayor ingreso y sus respectivos paises con sus rutas particulares.

## Analisis de la opcion 1: Rutas  de  importación  y  exportación.  Synergy  logistics  está considerando  la  posibilidad  de  enfocar  sus  esfuerzos  en  las  10  rutas  más demandadas. Acorde a los flujos de importación y exportación, ¿cuáles son esas 10 rutas? ¿le conviene implementar esa estrategia? ¿porqué? 

Para poder encontrar las rutas más demandas por rutas de exportación e importacion voy a sumar `total_value` por la columna `direction` y las rutas estan definidas como la combinacion única de pares `origin` y `destination`.
Esto se logra agrupando las columnas `origin` y `destination` para se sume la frecuencia de ordenes acorde al par único de rutas.

In [4]:
#Obten lista de rutas y cuantas veces se repiten 
df_rutas_con_frecuencia = synergy_data.\
    groupby(['origin', 'destination']).\
        size().\
        reset_index().\
        rename(columns={0:'frecuencia_d_ruta'}).\
    sort_values(by = 'frecuencia_d_ruta', ascending = False)
    
#filtra rutas que no ocurrieron    
ruta_data_mask = df_rutas_con_frecuencia.frecuencia_d_ruta > 0 #obten rutas que almenos hayan ocurrido una vez

clean_rutas_demandadas = df_rutas_con_frecuencia[ruta_data_mask]

clean_rutas_demandadas

,origin,destination,frecuencia_d_ruta
577,South Korea,Vietnam,497
444,Netherlands,Belgium,437
662,USA,Netherlands,436
355,Japan,Mexico,385
185,China,Mexico,351
...,...,...,...
226,France,Russia,13
350,Japan,India,8
205,France,Austria,7
111,Brazil,Germany,6


## Las 10 rutas más demandadas

El data frame anterior nos muestra las rutas mas demandas. Las 10 rutas más demandadas son:

In [5]:
clean_rutas_demandadas[:10].reset_index()

,index,origin,destination,frecuencia_d_ruta
0,577,South Korea,Vietnam,497
1,444,Netherlands,Belgium,437
2,662,USA,Netherlands,436
3,355,Japan,Mexico,385
4,185,China,Mexico,351
5,183,China,Japan,343
6,244,Germany,China,328
7,344,Japan,Brazil,306
8,246,Germany,France,299
9,557,South Korea,Japan,294


La pregunta que surge es si las 10 más demandadas son las 10 más valiosas, por lo que a continuación se calcula el valor total generado por ruta

In [6]:
#quiero obtener porcentajes por lo que calculo el total
total_historic_value_base_10x6 = (synergy_data 
     >> mutate(total_value_base_10x6 = _.total_value/1000000)).total_value_base_10x6.sum()

df_rutas_x_mean_y_total_value = (synergy_data 
     >> mutate(total_value_base_10x6 = _.total_value/1000000)
     >> select(_.origin, _.destination, _.total_value_base_10x6)
     >> group_by(_.origin, _.destination)
     >>  summarize(
         total_value =  _.total_value_base_10x6.sum()        
     )
     >> ungroup()                            
     >> mutate(
         porcentaje_del_total = _.total_value/total_historic_value_base_10x6*100
     )
     >> arrange(_.porcentaje_del_total)
     >> filter(_.total_value > 0)
    ) #solo muestra rutas hechas


df_rutas_valiosas_ordered = df_rutas_x_mean_y_total_value.sort_values(by = "porcentaje_del_total", ascending=False)

## Las 10 rutas más valiosas

El data frame anterior nos muestra las rutas por el valor generado de forma histórica. Las 10 rutas más valiosas son:

In [7]:
df_rutas_valiosas_ordered[0:10].reset_index()

,index,origin,destination,total_value,porcentaje_del_total
0,34,China,Mexico,12494.000,5.792538
1,26,Canada,Mexico,8450.000,3.917636
2,138,South Korea,Vietnam,6877.007,3.188356
3,33,China,Japan,5891.000,2.731218
4,90,Japan,Mexico,5829.000,2.702473
5,43,France,Belgium,5538.069,2.567590
6,53,France,United Kingdom,5427.000,2.516096
7,37,China,South Korea,4790.000,2.220766
8,134,South Korea,Japan,4741.000,2.198049
9,158,USA,Mexico,4710.000,2.183676


### Grafico un resumen de lo que encontre

In [8]:
df_50rutas_mas_demandas = clean_rutas_demandadas[:50].reset_index()
df_50rutas_mas_valiosas = df_rutas_valiosas_ordered[0:10].reset_index()

#test all routes
df_rutas_mas_demandas = clean_rutas_demandadas.reset_index()
df_rutas_mas_valiosas = df_rutas_valiosas_ordered.reset_index()

In [9]:
#rutas mas demandas
rutas_mas_demandadas = alt.Chart(df_50rutas_mas_demandas).mark_bar().encode(
    y= alt.Y('origin:N',
            sort = "-x"),
    x= alt.X('frecuencia_d_ruta:Q'),
    color='destination:N',
    tooltip=['origin', 'destination', 'frecuencia_d_ruta']
).interactive()

La gráfica anterior revela que el grueso de las rutas provienen de china, pero no todas las rutas de china son altamente valiosas. La tabla de rutas más frecuentes revela que la ruta corea del sur y vietnam es la más usada e incidentalmente es muy valiosa. Eso lo veremos en la gráfica siguiente

In [11]:
rutas_mas_valiosas = alt.Chart(df_50rutas_mas_valiosas).mark_bar().encode(
    y= alt.Y('origin:N',
            sort = "-x"),
    x= alt.X('total_value:Q'),
    color='destination:N',
    tooltip=['origin', 'destination', 'total_value']
).interactive()

rutas_mas_valiosas

alt.Chart(...)

#### A bigger picture

In [12]:
todas_rutas_mas_demandadas = alt.Chart(df_rutas_mas_demandas).mark_bar().encode(
    y= alt.Y('origin:N',
            sort = "-x"),
    x= alt.X('frecuencia_d_ruta:Q'),
    color= alt.Color('destination:N', sort = ['x']),
    tooltip=['origin', 'destination', 'frecuencia_d_ruta']
).interactive()

todas_rutas_mas_valiosas = alt.Chart(df_rutas_mas_valiosas).mark_bar().encode(
    y= alt.Y('origin:N',
            sort = "-x"),
    x= alt.X('total_value:Q'),
    color= alt.Color('destination:N', sort = ['x']),
    tooltip=['origin', 'destination', 'total_value']
).interactive()

#Bind the charts together
pts = alt.selection_multi(fields=['origin']) 

#when selecting demand route, se filtra solo a ver esa ruta en value
demand_rutas = todas_rutas_mas_demandadas.add_selection(pts)
value_rutas = todas_rutas_mas_valiosas.transform_filter(pts)

alt.hconcat(
    demand_rutas, value_rutas
)
#haz click afuera de la grafica para quitar el filtro y presiona shift para seleccionar varios

alt.HConcatChart(...)

## Análisis de la opción 2: Medio de transporte utilizado. ¿Cuáles son los 3 medios de transporte  más  importantes  para  Synergy  logistics  considerando  el  valor  de  las importaciones  y  exportaciones?  ¿Cuál  es  medio  de  transporte  que podrían reducir?

Para evaluar esta opción hay que calcular el valor total por medio de transporte

In [13]:
(synergy_data
  >> mutate(total_value_base_10x6 = _.total_value/1000000) #los valores que se ven son 10x6. millones 
  >> select(_.transport_mode, _.total_value_base_10x6)
  >> group_by(_.transport_mode)
  >> summarize(valor_total = _.total_value_base_10x6.sum())
).reset_index().sort_values(by="valor_total", ascending = False)

,index,transport_mode,valor_total
3,3,Sea,100530.622
1,1,Rail,43628.043
0,0,Air,38262.147
2,2,Road,33270.486


Ya veo el valor total que cada modo de transporte produce para la empresa, pero me interesa saber su contribución al valor total generado y al valor generado por año.

In [14]:
#calculo ingresos totales
ingresos_totales = synergy_data.total_value.sum()
#calculo el valor total por modo de transporte por year
synergy_Tvalue_per_modetransport = synergy_data.groupby(['transport_mode', 'year']).agg({'total_value': 'sum'})
#calculo el valor total por year
yearly_total = synergy_data.groupby(['year']).agg({'total_value': 'sum'})
#para obtener el peso porcentual por year, divido entre el total por year y el valor traido por modo de transporte
synergy_modoT_porcentual = (synergy_Tvalue_per_modetransport.div(yearly_total, level='year') * 100 )

synergy_modoT_anual = (synergy_modoT_porcentual.rename(columns={'total_value':'contribucion_anual_al_valor'})
)

synergy_modoT_anual['contribucion_anual_al_valor'] = synergy_modoT_anual['contribucion_anual_al_valor'].round(decimals = 2)
synergy_modoT_anual

contribucion_anual_al_valor
transport_mode year                               
Air            2015.0                        22.66
               2016.0                        10.76
               2017.0                        26.33
               2018.0                         6.62
               2019.0                        22.08
               2020.0                        18.03
Rail           2015.0                        14.32
               2016.0                        17.04
               2017.0                        24.64
               2018.0                        28.63
               2019.0                        11.44
               2020.0                        24.75
Road           2015.0                        26.55
               2016.0                        14.65
               2017.0                         5.93
               2018.0                        17.51
               2019.0                        21.33
               2020.0                         7.63
Sea            2015.0                        36.48
               2016.0                        57.55
               2017.0                        43.10
               2018.0                        47.24
               2019.0                        45.15
               2020.0                        49.59

### Grafico mis resultados

In [17]:
cincuenta = pd.DataFrame([{"threshold": 50}])
ochentaycinco = pd.DataFrame([{"threshold": 85}])

#its important to reset the index cause group DF dont work
bar_modeT = alt.Chart(synergy_modoT_anual.reset_index()).mark_bar().encode(
    y = alt.Y('year:O'),
    x = alt.X('contribucion_anual_al_valor:Q',
          scale=alt.Scale(domain=[0, 100])         
         ),
    color='transport_mode:N',
    tooltip=['transport_mode', 'year', 'contribucion_anual_al_valor']
).interactive()


rule_50 = alt.Chart(cincuenta).mark_rule().encode(
    x='threshold:Q'
)

rule_85 = alt.Chart(ochentaycinco).mark_rule().encode(
    x='threshold:Q'
)


(bar_modeT + rule_50 + rule_85) #.properties(width=600)

alt.LayerChart(...)

## Analisis de la opción 3: Valor total de importaciones y exportaciones. Si Synergy Logistics quisiera  enfocarse  en  los  países  que  le  generan  el  80%  del  valor  de  las exportaciones e importaciones  ¿en qué grupo de países debería enfocar sus esfuerzos?

Para considerar de manera global el valor que cada pais traer a la compañia, voy a agrupar por origen de envio y suma su valor historico. Despues, acumulo la suma para ver en que momento la suma acumulada llega a 80.

In [18]:
#calculo el valor total 
ingresos_totales = synergy_data.total_value.sum()

#para obtener el peso porcentual por year, divido entre el total por year y el valor traido por modo de transporte

value_per_pais = (synergy_data
 ).groupby('origin').agg({'total_value':'sum'}).sort_values(by = 'total_value', ascending = False)

value_per_pais['cumulative_sum'] = value_per_pais.\
 reset_index().set_index('origin').cumsum()

synergy_per_pais_value_porcentual = (value_per_pais.div(ingresos_totales) * 100 )

#lista de paises por contribucion
synergy_per_pais_value_porcentual

,total_value,cumulative_sum
origin,,
China,20.960533,20.960533
USA,10.963032,31.923565
Japan,9.292436,41.216001
France,9.240211,50.456213
South Korea,8.581777,59.037990
Germany,7.229421,66.267411
Russia,6.525066,72.792477
Canada,5.217178,78.009656
Italy,3.076009,81.085665


In [19]:
value_by_direction = synergy_data.groupby('direction').agg({'total_value':'sum'})

total = synergy_data.total_value.sum()
#para obtener el peso porcentual por year, divido entre el total por year y el valor traido por modo de transporte
direction_porcentual = (value_by_direction.div(total) * 100 )

#La tabla me dice que el grueso de sus ventas son exportaciones
direction_porcentual

,total_value
direction,
Exports,74.2558
Imports,25.7442


### Una gráfica para agregar claridad

In [20]:
contribucion_80 = pd.DataFrame([{"threshold": 80}])

#its important to reset the index cause group DF dont work
bar_contribucion = alt.Chart(synergy_per_pais_value_porcentual.reset_index()).mark_bar().encode(
    y = alt.Y('origin:O',
             sort = "-x"),
    x = alt.X('cumulative_sum:Q'),
    tooltip=['origin', 'cumulative_sum']
).interactive()

rule_80 = alt.Chart(contribucion_80).mark_rule().encode(
    x='threshold:Q'
)

(bar_contribucion + rule_80)


alt.LayerChart(...)

Rapidamente vemos el grupo de paises que le generan mayores ganancias a la empresa y en los que se deberia de enfocar.

## Conclusion: ¿Cuál opción u opciones es conveniente implementar como base en la estrategia?

La compañía debería fortalecer su cadena de logística para los países que le generan mayores ganancias, estos son:  China, USA, Japón, Francia, Corea del Sur, Alemania, Rusia, Canada, Italia y España. Esta conclusión se refuerza con la gráfica del análisis del opción 3 que muestra la contribución al valor total de importaciones y exportaciones que revela que sólo 7 países contribuyen al 80 % de sus ingresos históricos, siendo China el país más importante.

Los 3 medios de transporte más importante para la empresa por  orden de importancia son: transporte marítimo, transporte aéreo y transporte por rieles. El medio de transporte que se podría reducir es el terrestre.
